In [1]:
from mutagen.id3 import ID3, APIC

input_file = "/home/dgmastertemple/Documents/GitHub/create-audio-bible/projects/Ephesians - Tim Conway/export/verses/Ephesians 1:1 - Tim Conway.mp3"
cover_image = "/home/dgmastertemple/Documents/GitHub/create-audio-bible/sources/ill_be_honest.jpeg"

def add_album_art(input_file: str, cover_image: str):
    audio = ID3(input_file)
    with open(cover_image, 'rb') as image_file:
        image_data = image_file.read()
    mime='image/jpeg'
    if cover_image.endswith(".png"):
        mime='image/png'
    album_art = APIC(
        encoding=3,  # UTF-8
        mime=mime,
        type=3,  # Cover (front)
        desc='Cover (Front)',
        data=image_data
    )
    audio['APIC'] = album_art
    audio.save(input_file, v2_version=3)

add_album_art(input_file, cover_image)

In [3]:
from mutagen.mp3 import MP3
from mutagen.id3 import ID3, APIC

def set_album_cover(mp3_path, image_path):
    # Load the MP3 file
    audio = MP3(mp3_path, ID3=ID3)

    # Add ID3 tag if it doesn't exist
    try:
        audio.add_tags()
    except:
        pass

    # Open the image file
    with open(image_path, 'rb') as image_file:
        image_data = image_file.read()

    # Create the APIC frame (album cover)
    album_cover = APIC(
        encoding=3,  # UTF-8
        mime='image/jpeg',  # Or 'image/png' for PNG files
        type=3,  # 3 is for the cover image
        desc='Cover',
        data=image_data
    )

    # Remove existing APIC frames
    audio.tags.delall('APIC')

    # Add the new APIC frame
    audio.tags.add(album_cover)

    # Save the updated MP3 file
    audio.save()

# Usage example
mp3_file_path = 'path/to/your/song.mp3'
image_file_path = 'path/to/your/cover_image.jpg'

set_album_cover(input_file, cover_image)

In [8]:
from mutagen.mp3 import MP3

mp3_file_path = input_file

audio = MP3(mp3_file_path)
album_art = audio.tags.get('APIC', None)

if album_art:
    print(f"Album Art Type: {album_art.type}")
    print(f"Album Art MIME Type: {album_art.mime}")
    print(f"Album Art Description: {album_art.desc}")
    print(f"Album Art Data Length: {len(album_art.data)}")
else:
    print("No album art found in the MP3 file.")

No album art found in the MP3 file.


In [9]:
from mutagen.mp3 import MP3
from mutagen.id3 import ID3, APIC, error
from PIL import Image
import io

def set_album_cover(mp3_path, image_path):
    # Open the image file and ensure it's in the right format (JPEG or PNG)
    with open(image_path, 'rb') as img_file:
        img_data = img_file.read()

    # Determine the MIME type of the image
    img = Image.open(io.BytesIO(img_data))
    mime_type = Image.MIME.get(img.format)

    if not mime_type:
        raise ValueError(f"Unsupported image format: {img.format}")

    # Load the MP3 file
    audio = MP3(mp3_path, ID3=ID3)

    # Add ID3 tag if it doesn't exist
    if audio.tags is None:
        audio.add_tags()

    # Remove existing album art
    audio.tags.delall('APIC')

    # Add the album art
    audio.tags.add(
        APIC(
            encoding=3,  # 3 is for utf-8
            mime=mime_type,  # image/jpeg or image/png
            type=3,  # 3 is for the album cover
            desc=u'Cover',
            data=img_data
        )
    )

    # Save the changes to the MP3 file
    audio.save(v2_version=3)  # Save with ID3v2.3 tag

# Example usage
mp3_path = input_file
image_path = cover_image  # or image.png

set_album_cover(mp3_path, image_path)
